In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp

(CVXPY) Apr 18 03:59:44 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.5.2237). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Apr 18 03:59:44 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.5.2237). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [6]:
# Import data
df = pd.read_csv('data/stock_price.csv', index_col = 0, header=[0,1])
df.head()

Adj Close                                            \
                          AAPL       ABBV     AMZN       AVGO        CVX   
Date                                                                       
2013-03-22 00:00:00  14.247963  25.944092  12.8875  27.195913  80.051414   
2013-03-25 00:00:00  14.299474  26.177101  12.8010  26.983200  79.390778   
2013-03-26 00:00:00  14.224210  26.769625  13.0155  27.188309  79.919296   
2013-03-27 00:00:00  13.944750  26.729670  13.2650  27.074360  79.397400   
2013-03-28 00:00:00  13.654186  27.149092  13.3245  27.287069  78.492386   

                                                                            \
                          GOOG      GOOGL        JNJ        JPM         KO   
Date                                                                         
2013-03-22 00:00:00  20.182102  20.278028  60.786514  36.941341  29.217482   
2013-03-25 00:00:00  20.165415  20.261261  60.740765  36.744450  29.275867   
2013-03-26 00:00:00  20.234655  20.330832  61.632656  36.835316  29.691795   
2013-03-27 00:00:00  19.991568  20.086588  61.952839  36.176468  29.348839   
2013-03-28 00:00:00  19.780607  19.874624  62.151062  35.941704  29.509375   

                     ...    Volume                                          \
                     ...       MRK      MSFT      NVDA       PEP       PFE   
Date                 ...                                                     
2013-03-22 00:00:00  ...  10463232  28720900  23247600  14228500  24346346   
2013-03-25 00:00:00  ...  13746721  44154000  36207600   6873800  32376350   
2013-03-26 00:00:00  ...  12694110  27824300  23254000   5183700  33567476   
2013-03-27 00:00:00  ...  11497503  36047400  25494400   3601600  24848999   
2013-03-28 00:00:00  ...  17440502  55453800  34927600   5338100  29449919   

                                                                 
                    PGHD      TSLA      UNH         V       XOM  
Date                                                             
2013-03-22 00:00:00  NaN   6603000  5415900  18790400  10786600  
2013-03-25 00:00:00  NaN  35682000  5142500  19336800  12900800  
2013-03-26 00:00:00  NaN  27123000  6059700  17264800  12329600  
2013-03-27 00:00:00  NaN  19444500  8525800  13371200  12350800  
2013-03-28 00:00:00  NaN  17380500  5685400  12456400  17485200  

[5 rows x 138 columns]

In [13]:
df2 = df[['Adj Close']]
df2.columns = df2.columns.droplevel(0)
df2

,AAPL,ABBV,AMZN,AVGO,CVX,GOOG,GOOGL,JNJ,JPM,KO,...,MRK,MSFT,NVDA,PEP,PFE,PGHD,TSLA,UNH,V,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2013-03-22 00:00:00,14.247963,25.944092,12.887500,27.195913,80.051414,20.182102,20.278028,60.786514,36.941341,29.217482,...,30.582870,23.428345,2.899048,59.024342,18.816662,NaN,2.441333,46.713673,37.330597,58.387344
2013-03-25 00:00:00,14.299474,26.177101,12.801000,26.983200,79.390778,20.165415,20.261261,60.740765,36.744450,29.275867,...,30.436575,23.353706,2.882788,58.416367,18.670795,NaN,2.502000,47.056770,38.242363,58.210781
2013-03-26 00:00:00,14.224210,26.769625,13.015500,27.188309,79.919296,20.234655,20.330832,61.632656,36.835316,29.691795,...,30.924227,23.353706,2.903694,59.234486,18.962523,NaN,2.524000,47.734394,39.049168,58.936626
2013-03-27 00:00:00,13.944750,26.729670,13.265000,27.074360,79.397400,19.991568,20.086588,61.952839,36.176468,29.348839,...,30.722216,23.527863,2.938539,58.761631,18.989048,NaN,2.544000,48.566425,39.328999,59.230881
2013-03-28 00:00:00,13.654186,27.149092,13.324500,27.287069,78.492386,19.780607,19.874624,62.151062,35.941704,29.509375,...,30.791859,23.726900,2.980352,59.377113,19.134916,NaN,2.526000,49.072498,39.604153,58.923538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-15 00:00:00,152.990005,154.059998,96.199997,621.566284,153.800003,96.550003,96.110001,154.350006,128.259995,59.970001,...,107.629997,265.440002,242.279999,176.630005,40.279999,NaN,180.449997,465.429993,216.369995,101.620003
2023-03-16 00:00:00,155.850006,155.300003,100.040001,631.951538,154.289993,101.070000,100.320000,154.029999,130.750000,60.299999,...,107.339996,276.200012,255.410004,176.509995,40.369999,NaN,184.130005,472.019989,217.449997,101.029999
2023-03-17 00:00:00,155.000000,154.220001,98.949997,626.460999,152.339996,102.459999,101.620003,152.380005,125.809998,60.020000,...,104.099998,279.429993,257.250000,175.130005,40.099998,NaN,180.130005,469.500000,217.389999,99.839996


## Optimization Model

In [161]:
def NPM(r, his_r, x_tilde, theta, c):
    '''
    param r [ndarray]: vector of predicted price at period t
    param his_r [ndarray]: 2d vector of prices for the past W periods; his_r[i][j] = price 
    param x_tilde [ndarray]: investment weights at the end of period t-1
    param theta [float]: acceptable risk level in the range of (0, 1), smaller theta could lead to infeasible solution
    param c [float]: transacion cost in the range of (0, 1)

    '''
    M = len(r) # Number of assets in the portfolio
    W = len(his_r) # Number of recents prices used to measure risk
    
    # Define variables
    u = cp.Variable(M)
    v = cp.Variable(M)
    d = cp.Variable(W)
    g = cp.Variable(W)
    
    # Define Objective
    objective = cp.Maximize(r @ x_tilde + (x_tilde - c/2) @ u - (x_tilde + c/2) @ v)
    
    # Define constraints
    constraints = [cp.sum(d + g) <= W * theta,
                    cp.sum(u - v) == 0,
                    -(u - v + x_tilde) <= 0,
                    u - v + x_tilde <= 1,
                    u >= 0, v >= 0, d >= 0, g >= 0]
    for j in range(W):
        constraints += [(his_r[-1-j] - r) @ u + (his_r[-1-j] - r) @ v - d[j] <= (r - his_r[-1-j]) @ x_tilde,
                        (r - his_r[-1-j]) @ u + (r - his_r[-1-j]) @ v - d[j] <= (his_r[-1-j] - r) @ x_tilde]
        
    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve(verbose = True)
    print('problem status', problem.status)
#     constraints_value = [constraint.value for constraint in constraints]
#     print(constraints_value)
        
    return problem, u.value, v.value, d.value, g.value

Simple test case for NPM

In [162]:
# assets = ['AAPL', 'AMZN']
assets = df2.columns[:10]
specific_day = '2023-03-21 00:00:00'
w = 6 # Number of historical days
n = df2.index.get_loc(specific_day)

r = df2.loc[specific_day, assets] # The actual usage is that r should be the predicted price.
r = np.array(r)
print(r)

[159.27999878 156.77000427 100.61000061 636.75       159.30999756
 105.83999634 104.91999817 153.88999939 130.55000305  60.31999969]


In [163]:
his_r = df2.iloc[n-w:n, :][assets]
his_r = his_r.values
his_r

array([[150.47000122, 151.94999695,  92.43000031, 612.06463623,
        158.71000671,  91.66000366,  91.11000061, 153.05999756,
        131.25      ,  59.35472107],
       [152.58999634, 153.8500061 ,  94.87999725, 627.94042969,
        160.75999451,  94.25      ,  93.97000122, 153.91999817,
        134.61999512,  59.57304382],
       [152.99000549, 154.05999756,  96.19999695, 621.56628418,
        153.80000305,  96.55000305,  96.11000061, 154.3500061 ,
        128.25999451,  59.97000122],
       [155.8500061 , 155.30000305, 100.04000092, 631.95153809,
        154.28999329, 101.06999969, 100.31999969, 154.02999878,
        130.75      ,  60.29999924],
       [155.        , 154.22000122,  98.94999695, 626.46099854,
        152.33999634, 102.45999908, 101.62000275, 152.38000488,
        125.80999756,  60.02000046],
       [157.3999939 , 156.11999512,  97.70999908, 639.11004639,
        154.58000183, 101.93000031, 101.22000122, 153.88999939,
        127.13999939,  60.59999847]])

In [206]:
x_tilde = np.ones(10)/10
theta = 10
c = 0.001

In [207]:
problem, u, v, d, g = NPM(r, his_r, x_tilde, theta, c)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Apr 18 10:11:36 PM: Your problem has 32 variables, 20 constraints, and 0 parameters.
(CVXPY) Apr 18 10:11:36 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 18 10:11:36 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 18 10:11:36 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 18 10:11:36 PM: Compiling problem (target solver=ECOS).
(CVXPY) Apr 18 10:11:36 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

In [208]:
next_x = x_tilde + u - v
print('next_x', next_x)

next_x [0.1        0.1        0.1        0.10000001 0.1        0.1
 0.1        0.1        0.1        0.1       ]
